In [2]:
from allofplos import Corpus, starterdir

In [6]:
starterdir

'/home/balla/anaconda3/lib/python3.7/site-packages/allofplos/starter_corpus'

In [7]:
corpus = Corpus('/data02/allofplos')

In [96]:
len(corpus)

252785

In [9]:
display(corpus.random_article)

DOI: 10.1371/journal.pone.0192063
Title: Estimation of linkage disequilibrium and analysis of genetic diversity in Korean chicken lines

In [80]:
for article in corpus[10:20]:
    print("doi:", article.doi, "journal:", article.journal, "discipline",article.taxonomy)
    print("==================================================================================")

doi: 10.1371/journal.pone.0000011 journal: PLOS ONE discipline {'heading': [('Research Article',)], 'Discipline': [('Public Health and Epidemiology',)]}
doi: 10.1371/journal.pone.0000012 journal: PLOS ONE discipline {'heading': [('Research Article',)], 'Discipline': [('Computational Biology', 'Infectious Diseases/Epidemiology and Control of Infectious Diseases', 'Public Health and Epidemiology/Infectious Diseases')]}
doi: 10.1371/journal.pone.0000013 journal: PLOS ONE discipline {'heading': [('Research Article',)], 'Discipline': [('Molecular Biology',)]}
doi: 10.1371/journal.pone.0000014 journal: PLOS ONE discipline {'heading': [('Research Article',)], 'Discipline': [('Geriatrics', 'Neuroscience', 'Evolutionary Biology/Animal Behavior')]}
doi: 10.1371/journal.pone.0000015 journal: PLOS ONE discipline {'heading': [('Research Article',)], 'Discipline': [('Developmental Biology', 'Biotechnology/Bioengineering')]}
doi: 10.1371/journal.pone.0000016 journal: PLOS ONE discipline {'heading': [

In [90]:
print(Country(article).country)

None


In [86]:
article[doi]

NameError: name 'doi' is not defined

In [40]:
from peewee import Model, CharField, ForeignKeyField, TextField, \
    DateTimeField, BooleanField, IntegerField, IntegrityError

In [59]:
import allofplos.makedb as md

usage: ipykernel_launcher.py [-h] [--db DB] [--random RANDOM] [--starterdb]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/balla/.local/share/jupyter/runtime/kernel-d9a29ddc-f2c3-4127-b592-4ef1ef0e76b4.json


SystemExit: 2

/home/balla/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


NameError: name 'md' is not defined

In [28]:
article.elocation

'e10'

In [91]:
from bs4 import BeautifulSoup as bs

In [ ]:
'data02/allofplos'

In [94]:
with open('/data02/allofplos/journal.pone.0000011.xml', "r") as file:
    # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
    # Combine the lines in the list into a string
        content = "".join(content)
    
    
        bs_content = bs(content, "lxml")
#         subject = bs_content.find_all("subject")
#         abstract = bs_content.find("abstract")
#         title = bs_content.find("article-title")
#         author = bs_content.find_all("contrib")
#         affiliation = bs_content.findAll("aff")
#         a = bs_content.findAll("article")
#         title = bs_content.find("alt-title")
        year = bs_content.find("year")
        print(year)

<year>2006</year>


In [95]:
bs_content

<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE article PUBLIC "-//NLM//DTD Journal Publishing DTD v3.0 20080202//EN" "http://dtd.nlm.nih.gov/publishing/3.0/journalpublishing3.dtd">
<html><body><article article-type="research-article" dtd-version="3.0" xml:lang="EN" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink">
<front>
<journal-meta><journal-id journal-id-type="nlm-ta">PLoS ONE</journal-id><journal-id journal-id-type="publisher-id">plos</journal-id><journal-id journal-id-type="pmc">plosone</journal-id><!--===== Grouping journal title elements =====--><journal-title-group><journal-title>PLoS ONE</journal-title></journal-title-group><issn pub-type="epub">1932-6203</issn><publisher>
<publisher-name>Public Library of Science</publisher-name>
<publisher-loc>San Francisco, USA</publisher-loc>
</publisher></journal-meta>
<article-meta><article-id pub-id-type="publisher-id">06-PONE-RA-00083R1</article-id><article-id pub-id-type="doi">10.1371/journal

In [71]:
"""
Make a SQLite DB out of articles XML files
"""

import argparse
import datetime
import os
from itertools import islice

from tqdm import tqdm
import sqlite3

from peewee import Model, CharField, ForeignKeyField, TextField, \
    DateTimeField, BooleanField, IntegerField, IntegrityError
from playhouse.sqlite_ext import SqliteExtDatabase

from allofplos.corpus import Corpus
from allofplos.transformations import filename_to_doi, convert_country
from allofplos import starterdir
from allofplos.article import Article

journal_title_dict = {
    'PLOS ONE': 'PLOS ONE',
    'PLOS GENETICS': 'PLOS Genetics',
    'PLOS GENET': 'PLOS Genetics',
    'PLOS NEGLECTED TROPICAL DISEASES': 'PLOS Neglected Tropical Diseases',
    'PLOS NEGL TROP DIS': 'PLOS Neglected Tropical Diseases',
    'PLOS BIOLOGY': 'PLOS Biology',
    'PLOS BIOL': 'PLOS Biology',
    'PLOS CLINICAL TRIALS': 'PLOS Clinical Trials',
    'PLOS MEDICINE':'PLOS Medicine',
    'PLOS MEDICIN': 'PLOS Medicine',
    'PLOS MED': 'PLOS Medicine',
    'PLOS PATHOG': 'PLOS Pathogens',
    'PLOS PATHOGENS': 'PLOS Pathogens',
    'PLOS COMPUTATIONAL BIOLOGY': 'PLOS Computational Biology',
    'PLOS COMPUT BIOL': 'PLOS Computational Biology',
    'PLOS CLINICAL TRIALS': 'PLOS Clinical Trials',
}


parser= argparse.ArgumentParser()
parser.add_argument('--db', action='store', help=
                    'Name the db', default='ploscorpus.db')
parser.add_argument('--random', action='store', type=int, help=
                    'Number of articles in a random subset. '
                    'By default will use all articles')
parser.add_argument('--starterdb', action='store_true', help=
                    'Make the starter database', dest='starter')
args, unknown = parser.parse_known_args()

In [72]:
if os.path.isfile(args.db):
    os.remove(args.db)

if args.starter:
    if os.path.isfile('starter.db'):
        os.remove('starter.db')
    db = SqliteExtDatabase('starter.db')
else:
    db = SqliteExtDatabase(args.db)

class BaseModel(Model):
    class Meta:
        database = db

class Journal(BaseModel):
    """
    Journal table stores journals values. It is used in the PLOSArticle table.
    Fields:
    journal: Journal name. Based on the values of journal_title_dict
    """
    journal = CharField(unique=True)

class ArticleType(BaseModel):
    """
    ArticleType table stores article types used in PLOSArticle table.
    Fields:
    article_type: Article type such as Research Article, Retraction,
      Essay, Perspective and others.
    """
    article_type = CharField(unique=True)

class Country(BaseModel):
    """
    Country table stores countries that are related with the corresponding
      author.
    Fields:
    country: Country.
    """
    country = CharField(unique=True)

class Affiliations(BaseModel):
    """
    Affiliations table stores affiliations that are related with the
      corresponding author.
    Fields:
    affiliations: Author affiliations such as "Department of Public Health,
      University of Helsinki, Finland".
    """
    affiliations = CharField(unique=True)

class Subjects(BaseModel):
    """
    Subjects table stores subjects that are related to PLOSArticle table by
      using a link table (since an article may have multiple subjects)
    Fields:
    subjects: Subjects like "Molecular Biology", "Cell processes" and so on.
    """
    subjects = CharField(unique=True)

In [79]:
print(Journal(article).journal)

None


In [35]:
from allofplos.article import Article